In [1]:
import pandas as pd
import numpy as np
import gzip


class SOMToolBox_Parse:
    
    def __init__(self, filename):
        self.filename = filename
    
    def read_weight_file(self,):
        df = pd.DataFrame()
        if self.filename[-3:len(self.filename)] == '.gz':
            with gzip.open(self.filename, 'rb') as file:
                df, vec_dim, xdim, ydim = self._read_vector_file_to_df(df, file)
        else:
            with open(self.filename, 'rb') as file:
                df, vec_dim, xdim, ydim = self._read_vector_file_to_df(df, file)

        file.close()            
        return df.astype('float64'), vec_dim, xdim, ydim


    def _read_vector_file_to_df(self, df, file):
        xdim, ydim, vec_dim, position = 0, 0, 0, 0
        for byte in file:
            line = byte.decode('UTF-8')
            if line.startswith('$'):
                xdim, ydim, vec_dim = self._parse_vector_file_metadata(line, xdim, ydim, vec_dim)
                if xdim > 0 and ydim > 0 and len(df.columns) == 0:
                    df = pd.DataFrame(index=range(0, ydim * xdim), columns=range(0, vec_dim))
            else:
                if len(df.columns) == 0 or vec_dim == 0:
                    raise ValueError('Weight file has no correct Dimensional information.')
                position = self._parse_weight_file_data(line, position, vec_dim, df)
        return df, vec_dim, xdim, ydim


    def _parse_weight_file_data(self, line, position, vec_dim, df):
        splitted=line.split(' ')
        try:
            df.values[position] = list(np.array(splitted[0:vec_dim]).astype(float))
            position += 1
        except: raise ValueError('The input-vector file does not match its unit-dimension.') 
        return  position


    def _parse_vector_file_metadata(self, line, xdim, ydim, vec_dim):
        splitted = line.split(' ')
        if splitted[0] == '$XDIM':      xdim = int(splitted[1])
        elif splitted[0] == '$YDIM':    ydim = int(splitted[1])
        elif splitted[0] == '$VEC_DIM': vec_dim = int(splitted[1])
        return xdim, ydim, vec_dim 
        

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) 

import numpy as np
from scipy.spatial import distance_matrix, distance
from ipywidgets import Layout, HBox, Box, widgets, interact
import plotly.graph_objects as go
import networkx as nx


class SomViz:
    
    def __init__(self, weights=[], m=None, n=None):
        self.weights = weights
        self.m = m
        self.n = n

    def get_test_adj_mat(self, prob = 0.05):
        size = self.n * self.m
        test_mat = 1 * ((np.random.sample((size, size))) <= prob)
        #test_mat[np.tril_indices(size)] = 0
        return test_mat

    def umatrix(self, som_map=None, color="Viridis", interp = "best", title="", adj_mat=None):
        um =np.zeros((self.m *self.n, 1))
        neuron_locs = list()
        for i in range(self.m):
            for j in range(self.n):
                neuron_locs.append(np.array([i, j]))
        neuron_distmat = distance_matrix(neuron_locs,neuron_locs)

        for i in range(self.m * self.n):
            neighbor_idxs = neuron_distmat[i] <= 1
            neighbor_weights = self.weights[neighbor_idxs]
            um[i] = distance_matrix(np.expand_dims(self.weights[i], 0), neighbor_weights).mean()

        if som_map==None: return self.plot(um.reshape(self.m,self.n), color=color, interp=interp, title=title, adj_mat=adj_mat)
        else: som_map.data[0].z = um.reshape(self.m,self.n)

    def hithist(self, som_map=None, idata = [], color='RdBu', interp = "best", title="", adj_mat=None):
        hist = [0] *self.n *self.m
        for v in idata: 
            position =np.argmin(np.sqrt(np.sum(np.power(self.weights - v, 2), axis=1)))
            hist[position] += 1    
        
        if som_map==None: return self.plot(np.array(hist).reshape(self.m,self.n), color=color, interp=interp, title=title, adj_mat=adj_mat)
        else:  som_map.data[0].z = np.array(hist).reshape(self.m,self.n)


    def neighgraph(self, adj_mat, line_width=8, line_color='#fff', pos=None):

        # Delete lower triangle (and diagonal) of adjacency matrix
        adj_mat[np.tril_indices(adj_mat.shape[0])] = 0

        # Make networkx graph from input adjacency matrix
        G = nx.from_numpy_matrix(adj_mat, parallel_edges=True)

        if pos is None:
            pos = {}
            counter = 0
            for i in range(self.m):
                for j in range(self.n):
                    pos.update({counter : (j, i)})
                    counter += 1

        nx.set_node_attributes(G, pos, 'pos')

        # Convert nx graph to plotly trace data
        edge_x = []
        edge_y = []
        for edge in G.edges():
            x0, y0 = G.nodes[edge[0]]['pos']
            x1, y1 = G.nodes[edge[1]]['pos']
            edge_x.append(x0)
            edge_x.append(x1)
            edge_x.append(None)
            edge_y.append(y0)
            edge_y.append(y1)
            edge_y.append(None)

        return go.Scatter(
            x=edge_x, y=edge_y,
            line=dict(width=line_width, color=line_color),
            hoverinfo='none',
            mode='lines')

    def component_plane(self, som_map=None, component=0, color="Viridis", interp = "best", title=""):
        if som_map==None: return self.plot(self.weights[:,component].reshape(-1,self.n), color=color, interp=interp, title=title)   
        else:  som_map.data[0].z = self.weights[:,component].reshape(-1,n)

    def sdh(self, som_map=None, idata=[], sdh_type=1, factor=1, draw=True, color="Cividis", interp = "best", title=""):

        import heapq
        sdh_m = [0] *self.m *self.n

        cs=0
        for i in range(0,factor): cs += factor-i

        for vector in idata:
            dist = np.sqrt(np.sum(np.power(self.weights - vector, 2), axis=1))
            c = heapq.nsmallest(factor, range(len(dist)), key=dist.__getitem__)
            if (sdh_type==1): 
                for j in range(0,factor):  sdh_m[c[j]] += (factor-j)/cs # normalized
            if (sdh_type==2):
                for j in range(0,factor): sdh_m[c[j]] += 1.0/dist[c[j]] # based on distance
            if (sdh_type==3): 
                dmin = min(dist)
                for j in range(0,factor): sdh_m[c[j]] += 1.0 - (dist[c[j]]-dmin)/(max(dist)-dmin)  

        if som_map==None: return self.plot(np.array(sdh_m).reshape(-1,self.n), color=color, interp=interp, title=title)      
        else: som_map.data[0].z = np.array(sdh_m).reshape(-1,self.n)
        
    def project_data(self,som_m=None, idata=[], title=""):

        data_y = []
        data_x = []
        for v in idata:
            position =np.argmin(np.sqrt(np.sum(np.power(self.weights - v, 2), axis=1)))
            x,y = position % self.n, position // self.n
            data_x.extend([x])
            data_y.extend([y])
            
        if som_m!=None: som_m.add_trace(go.Scatter(x=data_x, y=data_y, mode = "markers", marker_color='rgba(255, 255, 255, 0.8)',))
    
    def time_series(self, som_m=None, idata=[], wsize=50, title=""): #not tested
             
        data_y = []
        data_x = [i for i in range(0,len(idata))]
        
        data_x2 = []
        data_y2 = []
        
        qmin = np.Inf
        qmax = 0
        
        step=1
        
        ps = []
        for v in idata:
            matrix = np.sqrt(np.sum(np.power(self.weights - v, 2), axis=1))
            position = np.argmin(matrix)
            qerror = matrix[position]
            if qmin>qerror: qmin = qerror
            if qmax<qerror: qmax = qerror
            ps.append((position, qerror))
       
        markerc=[]    
        for v in ps:
            data_y.extend([v[0]])
            rez = v[1]/qmax
 
            markerc.append('rgba(0, 0, 0, '+str(rez)+')') 
            
            x,y = v[0] % self.n, v[0] // self.n 
            if    x==0: y = np.random.uniform(low=y, high=y+.1)
            elif  x==self.m-1: y = np.random.uniform(low=y-.1, high=y)
            elif  y==0: x = np.random.uniform(low=x, high=x+.1)
            elif  y==self.n-1: x = np.random.uniform(low=x-.1, high=x)
            else: x,y = np.random.uniform(low=x-.1, high=x+.1), np.random.uniform(low=y-.1, high=y+.1)                           
            
            data_x2.extend([x])
            data_y2.extend([y]) 
    
        ts_plot = go.FigureWidget(go.Scatter(x=[], y=[], mode = "markers", marker_color=markerc, marker=dict(colorscale='Viridis', showscale=True, color=np.random.randn(500))))
        ts_plot.update_xaxes(range=[0, wsize])       

        
        ts_plot.data[0].x, ts_plot.data[0].y = data_x, data_y
        som_m.add_trace(go.Scatter(x=data_x2, y=data_y2, mode = "markers",))
  
        som_m.layout.height = 500
        ts_plot.layout.height = 500
        som_m.layout.width = 500
        ts_plot.layout.width = 1300
        
        return HBox([go.FigureWidget(som_m), go.FigureWidget(ts_plot)])
    
    def plot(self, matrix, color="Viridis", interp = "best", title="", adj_mat=None):
        heat_trace = go.Heatmap(z=matrix, zsmooth=interp, showscale=False, colorscale=color)
        data = [heat_trace]

        if adj_mat is not None:
            data = [heat_trace, self.neighgraph(adj_mat)]

        return go.FigureWidget(data = data, layout=go.Layout(width=700, height=700,title=title, title_x=0.5,))

In [3]:
import pandas as pd
import minisom as som
from sklearn import datasets, preprocessing
from NeighbourhoodGraph import NeighbourhoodGraph as NG
#interp: False, 'best', 'fast', 
#color = 'viridis': https://plotly.com/python/builtin-colorscales/
min_max_scaler = preprocessing.MinMaxScaler()

In [4]:
#############################
######## miniSOM ############1/0
#############################
m=10
n=10

# Pre-processing 
ivec_iris_online = datasets.load_iris().data
ivec_iris_online = min_max_scaler.fit_transform(ivec_iris_online)   # optional
idim = ivec_iris_online.shape[0]

# Train
smap_iris_new = som.MiniSom(m, n, ivec_iris_online.shape[1], sigma=0.8, learning_rate=0.7)
%time smap_iris_new.train_random(ivec_iris_online, 10000, verbose=False)

sweights = smap_iris_new._weights.reshape(-1,4)
sdim = sweights.shape[0]
smap_x = n
smap_y = m

CPU times: user 413 ms, sys: 13.5 ms, total: 427 ms
Wall time: 414 ms


Visualization

In [5]:
viz = SomViz(sweights, m, n)
color='magma'
interp='best'
data_title='miniSOM'

idata = ivec_iris_online
k_list = [1,2,3,4]
r_list = [0.01,0.05,0.1,0.15]
r_list = np.around(np.mean(idata)*np.array(r_list), decimals=2) # scale radii to input data

# Prepare Neighbourhood Graph
ng2 = NG(viz.weights, viz.n, viz.m, input_data=idata)

## U-Matrix
display(viz.umatrix(color=color, interp=interp, title='{}: U-Matrix'.format(data_title)))

## U-Matrix with Neighbourhood Graph
# knn
for k in k_list:
    display(viz.umatrix(color=color, interp=interp, title='{}: U-Matrix + {}-NN Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(knn=k)))
# radius
for k in r_list:
    display(viz.umatrix(color=color, interp=interp, title='{}: U-Matrix + {}-Radius Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(radius=k)))

## Hit-Hist with Neighbourhood Graph
# knn
for k in k_list:
    display(viz.hithist(idata=idata, color=color, interp=interp, title='{}: Hit hist + {}-NN Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(knn=k)))
# radius
for k in r_list:
    display(viz.hithist(idata=idata, color=color, interp=interp, title='{}: Hit hist + {}-Radius Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(radius=k)))


FigureWidget({
    'data': [{'colorscale': [[0.0, '#000004'], [0.1111111111111111, '#180f3d'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#000004'], [0.1111111111111111, '#180f3d'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#000004'], [0.1111111111111111, '#180f3d'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#000004'], [0.1111111111111111, '#180f3d'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#000004'], [0.1111111111111111, '#180f3d'],
               …

...

FigureWidget({
    'data': [{'colorscale': [[0.0, '#000004'], [0.1111111111111111, '#180f3d'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#000004'], [0.1111111111111111, '#180f3d'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#000004'], [0.1111111111111111, '#180f3d'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#000004'], [0.1111111111111111, '#180f3d'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#000004'], [0.1111111111111111, '#180f3d'],
               …

In [6]:
##########################################
######## read from SOMToolBox ############
##########################################
ivec_iris_from_file = SOMToolBox_Parse('../input/iris/iris.vec')
idata, idim, _, _ = ivec_iris_from_file.read_weight_file()
idata = idata.to_numpy()

smap_iris_from_file = SOMToolBox_Parse('../input/iris/iris.wgt.gz')
sweights, sdim, smap_x, smap_y = smap_iris_from_file.read_weight_file()
sweights = sweights.values.reshape(-1,sdim)

Visualization

In [7]:
viz = SomViz(sweights, smap_y, smap_x)

color='viridis'
interp=False
data_title='SOMToolBox'

k_list = [1,2,3,4]
r_list = [0.01,0.05,0.1,0.15]
r_list = np.around(np.mean(idata)*np.array(r_list), decimals=2)

# Prepare Neighbourhood Graph
ng2 = NG(viz.weights, viz.n, viz.m, input_data=idata)

## U-Matrix
display(viz.umatrix(color=color, interp=interp, title='{}: U-Matrix'.format(data_title)))

## U-Matrix with Neighbourhood Graph
# knn
for k in k_list:
    display(viz.umatrix(color=color, interp=interp, title='{}: U-Matrix + {}-NN Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(knn=k)))
# radius
for k in r_list:
    display(viz.umatrix(color=color, interp=interp, title='{}: U-Matrix + {}-Radius Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(radius=k)))

## Hit-Hist with Neighbourhood Graph
# knn
for k in k_list:
    display(viz.hithist(idata=idata, color=color, interp=interp, title='{}: Hit hist + {}-NN Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(knn=k)))
# radius
for k in r_list:
    display(viz.hithist(idata=idata, color=color, interp=interp, title='{}: Hit hist + {}-Radius Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(radius=k)))




FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

...

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

## Chain Link Dataset

### Train new SOM

In [8]:
#############################
######## ChainLink ##########
#############################
m=40
n=20

# Pre-processing 
ivec = SOMToolBox_Parse('../input/chainlink/chainlink.vec') # set data here
idata, idim, _, _ = ivec.read_weight_file()
idata_chainlink = idata.to_numpy()

idata_chainlink = min_max_scaler.fit_transform(idata_chainlink)   # optional
idim = idata_chainlink.shape[0]

# Train
smap_chainlink = som.MiniSom(m, n, idata_chainlink.shape[1], sigma=0.8, learning_rate=0.7)
%time smap_chainlink.train_random(idata_chainlink, 10000, verbose=False)

sweights = smap_chainlink._weights.reshape(-1,3)
sdim = sweights.shape[0]
smap_x = n
smap_y = m

CPU times: user 939 ms, sys: 0 ns, total: 939 ms
Wall time: 939 ms


In [9]:
# Visualization
# Choose setup

viz = SomViz(sweights, smap_y, smap_x)

color='viridis'
interp=False
data_title='SOMToolBox'

k_list = [1,2,3,4]
r_list = [0.01,0.05,0.1,0.15]
r_list = np.around(np.mean(idata_chainlink)*np.array(r_list), decimals=2) # scale radii to input data

# Prepare Neighbourhood Graph
%time ng2 = NG(viz.weights, viz.n, viz.m, input_data=idata_chainlink)

# U-Matrix
display(viz.umatrix(color=color, interp=interp, title='{}: U-Matrix'.format(data_title)))

# U-Matrix with Neighbourhood Graph
## knn
for k in k_list:
    display(viz.umatrix(color=color, interp=interp, title='{}: U-Matrix + {}-NN Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(knn=k)))
## radius
for k in r_list:
    display(viz.umatrix(color=color, interp=interp, title='{}: U-Matrix + {}-Radius Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(radius=k)))

# Hit-Hist with Neighbourhood Graph
## knn
for k in k_list:
    display(viz.hithist(idata=idata_chainlink, color=color, interp=interp, title='{}: Hit hist + {}-NN Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(knn=k)))
## radius
for k in r_list:
    display(viz.hithist(idata=idata_chainlink, color=color, interp=interp, title='{}: Hit hist + {}-Radius Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(radius=k)))





CPU times: user 21 ms, sys: 164 µs, total: 21.1 ms
Wall time: 20.4 ms


FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

...

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

### Load Trained SOM

In [10]:
##########################################
######## read from SOMToolBox ############
##########################################

smap_chainlink_from_file = SOMToolBox_Parse('../input/chainlink/maps/chainlink.wgt.gz')
sweights, sdim, smap_x, smap_y = smap_chainlink_from_file.read_weight_file()
sweights = sweights.values.reshape(-1,sdim)

In [11]:
viz = SomViz(sweights, smap_y, smap_x)

color='viridis'
interp=False
data_title='SOMToolBox'

k_list = [1,2,3,4]
r_list = [0.01,0.05,0.1,0.15]
r_list = np.around(np.mean(idata_chainlink)*np.array(r_list), decimals=2)

# Prepare Neighbourhood Graph
ng2 = NG(viz.weights, viz.n, viz.m, input_data=idata_chainlink)

## U-Matrix
display(viz.umatrix(color=color, interp=interp, title='{}: U-Matrix'.format(data_title)))

## U-Matrix with Neighbourhood Graph
# knn
for k in k_list:
    display(viz.umatrix(color=color, interp=interp, title='{}: U-Matrix + {}-NN Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(knn=k)))
# radius
for k in r_list:
    display(viz.umatrix(color=color, interp=interp, title='{}: U-Matrix + {}-Radius Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(radius=k)))

## Hit-Hist with Neighbourhood Graph
# knn
for k in k_list:
    display(viz.hithist(idata=idata_chainlink, color=color, interp=interp, title='{}: Hit hist + {}-NN Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(knn=k)))
# radius
for k in r_list:
    display(viz.hithist(idata=idata_chainlink, color=color, interp=interp, title='{}: Hit hist + {}-Radius Neighbourhood Graph'.format(data_title, k), adj_mat=ng2.create_graph(radius=k)))



FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

...

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …

FigureWidget({
    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
               …